In [1]:
import pandas as pd
import numpy as np
import glob
import os

In [2]:
#import the weather files for top 20 origin airports of 2017 obtained from:
# https://mesonet.agron.iastate.edu/ASOS/
path = './Data/Weather/'
all_files = glob.glob(os.path.join(path, "*.csv"))     # advisable to use os.path.join as this makes concatenation OS independent

df_from_each_file = (pd.read_csv(f, parse_dates=['valid']) for f in all_files)
concatenated_df = pd.concat(df_from_each_file)

In [3]:
weather_df = concatenated_df

In [4]:
weather_df.head()

,station,network,valid,precip_in
0,LAX,CA_ASOS,2016-12-31 22:00:00,0.0
1,LAX,CA_ASOS,2016-12-31 23:00:00,0.0
2,LAX,CA_ASOS,2017-01-01 00:00:00,0.0
3,LAX,CA_ASOS,2017-01-01 01:00:00,0.0
4,LAX,CA_ASOS,2017-01-01 02:00:00,0.0


In [5]:
#export raw, concoctanated weather file for 2017 top 20 airports
weather_df.to_csv('./Data/Weather/2017Weather.csv', index = False, encoding = 'utf-8')

In [5]:
#import the 2017 Data and clean it
df = pd.read_csv('./Data/2017ALL.csv', parse_dates=['FlightDate'])


df = df.dropna(subset=['Tail_Number','DepTime','DepDelay','DepartureDelayGroups','TaxiOut',
                      'WheelsOff','WheelsOn','TaxiIn','ArrTime','ArrDelay','CRSElapsedTime',
                      'ActualElapsedTime','AirTime'])

df = df.fillna(0)

In [6]:
#Let's remove the features that will not be available for predicting flights
df = df[['Quarter', 'Month', 'DayofMonth', 'DayOfWeek', 'FlightDate',
       'Reporting_Airline', 'Tail_Number', 'Flight_Number_Reporting_Airline',
       'Origin', 'OriginState', 'DestState', 'Dest', 'CRSDepTime', 'CRSArrTime', 'CRSElapsedTime',
       'ArrDelay', 'Distance', 'DistanceGroup',]]
df = df.reindex()

In [7]:
#since the DEP_TIME is presented as a float we need to convert it to a Timestamp
def convert_to_time (datatype):
    '''This method will take a float as input and returns a datetime.time object'''
    to_string = str(int(datatype))
    #check length of string and add leading zeros if lenght is less than 4
    while len(to_string) < 4:
        to_string = '0' + to_string
    #check if hour starts with 24 then change to 00
    if to_string[:2] == '24':
        to_string = '00' + to_string[2:]
    #obj = time.strptime(to_string, '%H%M')
    
    to_string = to_string[:2] + ':' + to_string[2:] 
    datetime = pd.to_datetime(to_string)
    datetime = pd.to_datetime(to_string, format='%H:%M')
    time = datetime.time()
    return time
    
    #datetime = time.strftime("%H:%M", obj)
    #return datetime
    
#We now need to create a new Timestamp column incorporating the information from FL_DATE and DEP_TIME
def combine_datetime (dataframe):
    '''This function will extract needed data from two columns and combine into a single Timestamp object'''
    year = dataframe[0].year
    month = dataframe[0].month
    day = dataframe[0].day
    hour = dataframe[1].hour
    minute = dataframe[1].minute
    second = dataframe[1].second
    
    datetime = pd.Timestamp(year, month, day, hour, minute, second)
    return datetime

In [8]:
combined = df[['FlightDate','CRSDepTime']]

In [9]:
#convert DEP_TIME to timestamp
combined['CRSDepTime'] = df.CRSDepTime.apply(convert_to_time)

/Users/dmitriykats/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [10]:
#apply the combine_datetime function to optimized_df and create a new column 'FL_TIME'
df['FL_TIME'] = combined.apply(combine_datetime, axis=1)

In [11]:
df['FL_TIME_NEW'] = df.FL_TIME.apply(lambda x: x.replace(minute=0))

In [12]:
df.head()

,Quarter,Month,DayofMonth,DayOfWeek,FlightDate,Reporting_Airline,Tail_Number,Flight_Number_Reporting_Airline,Origin,OriginState,DestState,Dest,CRSDepTime,CRSArrTime,CRSElapsedTime,ArrDelay,Distance,DistanceGroup,FL_TIME,FL_TIME_NEW
0,2,5,6,6,2017-05-06,WN,N7824A,4652,SJC,CA,CA,SAN,650,810,80.0,2.0,417.0,2,2017-05-06 06:50:00,2017-05-06 06:00:00
1,2,5,6,6,2017-05-06,WN,N8522P,4971,SJC,CA,CA,SAN,1710,1830,80.0,-7.0,417.0,2,2017-05-06 17:10:00,2017-05-06 17:00:00
2,2,5,6,6,2017-05-06,WN,N8617E,5113,SJC,CA,CA,SAN,1100,1220,80.0,0.0,417.0,2,2017-05-06 11:00:00,2017-05-06 11:00:00
3,2,5,6,6,2017-05-06,WN,N450WN,5150,SJC,CA,CA,SAN,2030,2150,80.0,1.0,417.0,2,2017-05-06 20:30:00,2017-05-06 20:00:00
4,2,5,6,6,2017-05-06,WN,N498WN,5711,SJC,CA,CA,SAN,1325,1445,80.0,-2.0,417.0,2,2017-05-06 13:25:00,2017-05-06 13:00:00


In [13]:
#combine the station name and datetime for easier / faster merging with flight data
weather_df_new = pd.DataFrame()
weather_df_new['combined'] = weather_df['station'] + weather_df['valid'].astype(str)
weather_df_new['precip'] = weather_df['precip_in']

In [14]:
weather_df_new.head()

,combined,precip
0,LAX2016-12-31 22:00:00,0.0
1,LAX2016-12-31 23:00:00,0.0
2,LAX2017-01-01 00:00:00,0.0
3,LAX2017-01-01 01:00:00,0.0
4,LAX2017-01-01 02:00:00,0.0


In [43]:
sum_precip = []
for row in range(len(weather_df_new.precip) - 4):
    rows_sum = sum(weather_df_new.iloc[row:row + 4, 1].values)
    sum_precip.append(rows_sum)
    

In [50]:
sum_precip.insert(0,0)
sum_precip.insert(0,0)
sum_precip.insert(0,0)
sum_precip.insert(0,0)

In [52]:
weather_df_new['precip_sum'] = sum_precip

In [56]:
weather_df_new = weather_df_new.drop(columns=['precip'])

In [57]:
#do the same in the flight data 
df['combined'] =  df['Origin'] + df['FL_TIME_NEW'].astype(str)

In [58]:
df.head()

,Quarter,Month,DayofMonth,DayOfWeek,FlightDate,Reporting_Airline,Tail_Number,Flight_Number_Reporting_Airline,Origin,OriginState,...,Dest,CRSDepTime,CRSArrTime,CRSElapsedTime,ArrDelay,Distance,DistanceGroup,FL_TIME,FL_TIME_NEW,combined
0,2,5,6,6,2017-05-06,WN,N7824A,4652,SJC,CA,...,SAN,650,810,80.0,2.0,417.0,2,2017-05-06 06:50:00,2017-05-06 06:00:00,SJC2017-05-06 06:00:00
1,2,5,6,6,2017-05-06,WN,N8522P,4971,SJC,CA,...,SAN,1710,1830,80.0,-7.0,417.0,2,2017-05-06 17:10:00,2017-05-06 17:00:00,SJC2017-05-06 17:00:00
2,2,5,6,6,2017-05-06,WN,N8617E,5113,SJC,CA,...,SAN,1100,1220,80.0,0.0,417.0,2,2017-05-06 11:00:00,2017-05-06 11:00:00,SJC2017-05-06 11:00:00
3,2,5,6,6,2017-05-06,WN,N450WN,5150,SJC,CA,...,SAN,2030,2150,80.0,1.0,417.0,2,2017-05-06 20:30:00,2017-05-06 20:00:00,SJC2017-05-06 20:00:00
4,2,5,6,6,2017-05-06,WN,N498WN,5711,SJC,CA,...,SAN,1325,1445,80.0,-2.0,417.0,2,2017-05-06 13:25:00,2017-05-06 13:00:00,SJC2017-05-06 13:00:00


In [59]:
copy_df = df.copy()

In [60]:
copy_df.head()

,Quarter,Month,DayofMonth,DayOfWeek,FlightDate,Reporting_Airline,Tail_Number,Flight_Number_Reporting_Airline,Origin,OriginState,...,Dest,CRSDepTime,CRSArrTime,CRSElapsedTime,ArrDelay,Distance,DistanceGroup,FL_TIME,FL_TIME_NEW,combined
0,2,5,6,6,2017-05-06,WN,N7824A,4652,SJC,CA,...,SAN,650,810,80.0,2.0,417.0,2,2017-05-06 06:50:00,2017-05-06 06:00:00,SJC2017-05-06 06:00:00
1,2,5,6,6,2017-05-06,WN,N8522P,4971,SJC,CA,...,SAN,1710,1830,80.0,-7.0,417.0,2,2017-05-06 17:10:00,2017-05-06 17:00:00,SJC2017-05-06 17:00:00
2,2,5,6,6,2017-05-06,WN,N8617E,5113,SJC,CA,...,SAN,1100,1220,80.0,0.0,417.0,2,2017-05-06 11:00:00,2017-05-06 11:00:00,SJC2017-05-06 11:00:00
3,2,5,6,6,2017-05-06,WN,N450WN,5150,SJC,CA,...,SAN,2030,2150,80.0,1.0,417.0,2,2017-05-06 20:30:00,2017-05-06 20:00:00,SJC2017-05-06 20:00:00
4,2,5,6,6,2017-05-06,WN,N498WN,5711,SJC,CA,...,SAN,1325,1445,80.0,-2.0,417.0,2,2017-05-06 13:25:00,2017-05-06 13:00:00,SJC2017-05-06 13:00:00


In [61]:
weather_df_new.head()

,combined,precip_sum
0,LAX2016-12-31 22:00:00,0.0
1,LAX2016-12-31 23:00:00,0.0
2,LAX2017-01-01 00:00:00,0.0
3,LAX2017-01-01 01:00:00,0.0
4,LAX2017-01-01 02:00:00,0.0


In [ ]:
#mapping the weather data to the flight dataframe

In [62]:
weather_df_new = weather_df_new.set_index('combined')

In [63]:
weather_df_new.head()

,precip_sum
combined,
LAX2016-12-31 22:00:00,0.0
LAX2016-12-31 23:00:00,0.0
LAX2017-01-01 00:00:00,0.0
LAX2017-01-01 01:00:00,0.0
LAX2017-01-01 02:00:00,0.0


In [64]:
weather_dict = weather_df_new.to_dict()

In [65]:
weather_dict = weather_dict['precip_sum']

In [66]:
copy_df['precip_sum'] = copy_df.combined.map(weather_dict)

In [67]:
copy_df.head()

,Quarter,Month,DayofMonth,DayOfWeek,FlightDate,Reporting_Airline,Tail_Number,Flight_Number_Reporting_Airline,Origin,OriginState,...,CRSDepTime,CRSArrTime,CRSElapsedTime,ArrDelay,Distance,DistanceGroup,FL_TIME,FL_TIME_NEW,combined,precip_sum
0,2,5,6,6,2017-05-06,WN,N7824A,4652,SJC,CA,...,650,810,80.0,2.0,417.0,2,2017-05-06 06:50:00,2017-05-06 06:00:00,SJC2017-05-06 06:00:00,NaN
1,2,5,6,6,2017-05-06,WN,N8522P,4971,SJC,CA,...,1710,1830,80.0,-7.0,417.0,2,2017-05-06 17:10:00,2017-05-06 17:00:00,SJC2017-05-06 17:00:00,NaN
2,2,5,6,6,2017-05-06,WN,N8617E,5113,SJC,CA,...,1100,1220,80.0,0.0,417.0,2,2017-05-06 11:00:00,2017-05-06 11:00:00,SJC2017-05-06 11:00:00,NaN
3,2,5,6,6,2017-05-06,WN,N450WN,5150,SJC,CA,...,2030,2150,80.0,1.0,417.0,2,2017-05-06 20:30:00,2017-05-06 20:00:00,SJC2017-05-06 20:00:00,NaN
4,2,5,6,6,2017-05-06,WN,N498WN,5711,SJC,CA,...,1325,1445,80.0,-2.0,417.0,2,2017-05-06 13:25:00,2017-05-06 13:00:00,SJC2017-05-06 13:00:00,NaN


In [69]:
copy_df.Month.count()

5579410

In [70]:
df.Month.count()

5579410

In [72]:
weather_df_new.sort_values('precip_sum', ascending=False)

,precip_sum
combined,
IAH2017-08-27 08:00:00,4.86
IAH2017-08-27 08:00:00,4.86
IAH2017-08-27 08:00:00,4.86
IAH2017-08-27 08:00:00,4.86
IAH2017-08-27 02:00:00,4.83
IAH2017-08-27 02:00:00,4.83
IAH2017-08-27 02:00:00,4.83
IAH2017-08-27 02:00:00,4.83
IAH2017-08-27 03:00:00,4.52


In [73]:
#let's spotcheck the imported data
copy_df[(copy_df.Origin == 'IAH') & (copy_df.FL_TIME_NEW == '2017-08-27 08:00:00')]

,Quarter,Month,DayofMonth,DayOfWeek,FlightDate,Reporting_Airline,Tail_Number,Flight_Number_Reporting_Airline,Origin,OriginState,...,CRSDepTime,CRSArrTime,CRSElapsedTime,ArrDelay,Distance,DistanceGroup,FL_TIME,FL_TIME_NEW,combined,precip_sum
3964668,3,8,27,7,2017-08-27,AA,N3HHAA,1051,IAH,TX,...,847,1005,78.0,13.0,224.0,1,2017-08-27 08:47:00,2017-08-27 08:00:00,IAH2017-08-27 08:00:00,4.86
4072547,3,8,27,7,2017-08-27,UA,N806UA,527,IAH,TX,...,851,1129,158.0,281.0,925.0,4,2017-08-27 08:51:00,2017-08-27 08:00:00,IAH2017-08-27 08:00:00,4.86
4158826,3,8,27,7,2017-08-27,DL,N955DL,2258,IAH,TX,...,820,1119,119.0,59.0,689.0,3,2017-08-27 08:20:00,2017-08-27 08:00:00,IAH2017-08-27 08:00:00,4.86
4230635,3,8,27,7,2017-08-27,OO,N898SK,3529,IAH,TX,...,800,1018,198.0,10.0,1195.0,5,2017-08-27 08:00:00,2017-08-27 08:00:00,IAH2017-08-27 08:00:00,4.86


In [74]:
#export the dataframe as a csv to model
copy_df.to_csv('./Data/2017NEW.csv', index = False, encoding = 'utf-8')